# Course 3: Bokeh, interaction and web


### Links to resources:

- https://docs.bokeh.org/en/latest/docs/user_guide/interaction.html

Note: Throughout this notebook, we are using the specific feature that allows direct display within the notebook rather than in a web page:


In [1]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

# 1. Toolbar

https://docs.bokeh.org/en/latest/docs/user_guide/interaction/tools.html#plot-tools

Bokeh comes with a toolbar, Plot Tools, that provides access to interactive features:

- Pan Tool: Move the curve within the graph
- Box Zoom Tool: Zoom into a specific area of the graph
- Wheel Zoom Tool: Zoom using the mouse wheel
- Save Tool: Export the current view as PNG
- Reset Tool: Revert to the default configuration
- Help Symbol: Learn more about the tools
- Bokeh logo

There are many options to customize the tools, such as specifying desired tools, changing the toolbar's position (see documentation).


In [2]:
# Example 1: Toolbar positioned below

p = figure(width=400, height=400, title=None, toolbar_location="below")

p.circle([1, 2, 3, 4, 5], [2, 5, 8, 2, 7], size=10)
show(p)

In [3]:
# Example 2: Specify the list of tools to use, and the active ones.

from bokeh.plotting import figure, show

p = figure(width=400, height=400, title=None,
           tools="pan,wheel_zoom,reset", active_scroll="wheel_zoom")


p.circle([1, 2, 3, 4, 5], [2, 5, 8, 2, 7], size=10)
show(p)

In [4]:
# Example 3: Hide all tools and Bokeh logo

from bokeh.plotting import figure, show

# tools = "" removes all tools
p = figure(width=400, height=400, title=None, tools="")

p.toolbar.logo = None  # Remove the logo

p.circle([1, 2, 3, 4, 5], [2, 5, 8, 2, 7], size=10)
show(p)

# 2. Interactive Legend

Bokeh allows making a legend clickable to partially or completely hide entries.

Simply specify the "click_policy" option of the legend. With the "hide" parameter, the series is hidden when clicked. With the "mute" parameter, the series remains visible but in a faded color.

The following example presents two configurations.


In [5]:
from bokeh.plotting import figure, show, ColumnDataSource
import pandas as pd
from bokeh.palettes import Set1
from bokeh.layouts import row

# Data preparation
df = pd.DataFrame(columns=["x", "y1", "y2", "y3"])
df["x"] = [3, 4, 5, 6, 7]
df["y1"] = [12, 15, 16, 18, 20]
df["y2"] = [14, 15, 13, 12, 14]
df["y3"] = [14, 10, 8, 7, 6]

data = ColumnDataSource(df)

# Construct the first plot with a clickable legend
p1 = figure()
p1.title.text = "Click on the legend to hide entries"
for name, color in zip(["y1", "y2", "y3"], Set1[3]):
    p1.line("x", name, source=data, line_width=2,
            color=color, alpha=0.8, legend_label=name)

p1.legend.click_policy = "hide"

# Construct the second plot with a clickable legend
p2 = figure()
p2.title.text = 'Click on the legend to mute entries'
for name, color in zip(["y1", "y2", "y3"], Set1[3]):
    p2.line("x", name, source=data, line_width=2, color=color, alpha=0.8, legend_label=name,
            muted_color=color, muted_alpha=0.2)

p2.legend.click_policy = "mute"

layout = row(p1, p2)

show(layout)

# 3. Interactive Widgets

Documentation: https://docs.bokeh.org/en/latest/docs/user_guide/interaction/widgets.html

Bokeh offers a variety of widgets - tools for interacting with the content of the graph. Here is a list of available objects:

- Button: button
- Color Picker: colorpicker
- Spinner: spinner
- Checkbox Group: checkboxgroup
- Data Table: datatable
- Dropdown: dropdown
- Radio Group: radiogroup
- Slider: slider
- ...

The general principle of use is:

1. Create the widget - create a button, for example.
2. Include it in the web page using layout tools (rows and columns).
3. Code the widget's behavior by linking it to a `Callback function`. This code, called `Callback function`, is executed dynamically on the web page when the widget is activated.

Bokeh offers two ways to code `Callback function`:

- Use a predefined method in Python: `js_link`
- Directly code the JavaScript: `Custom_js`

A callback is triggered when some events occurs:

- Most Bokeh objects have a `.js_on_change` property whenever the state of the object changes.
- Some widget also have a `js_on_event` property.

## 3.1 Using js_link

Consider the following example:


In [6]:

from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show

# Basic data
x = [1, 2, 3, 4, 5, 6]
y = [15, 16, 15, 15, 17, 18]
data = ColumnDataSource(data={'x': x, 'y': y})

# Construct the graph
plt = figure(title="Application with colors")
points = plt.circle(x='x', y='y', source=data,
                    fill_color=None, line_color='red', size=15)

show(plt)

We will customize this chart to add a color picker and a tool to choose the circle size.

First step: create the `ColorPicker` and `Spinner` tools and organize them in the layout.


In [7]:

from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.models import ColorPicker, Spinner
from bokeh.layouts import row, column

x = [1, 2, 3, 4, 5, 6]
y = [15, 16, 15, 15, 17, 18]
data = ColumnDataSource(data={'x': x, 'y': y})
plt = figure(title="Application with colors")
points = plt.circle(x='x', y='y', source=data,
                    fill_color=None, line_color='red', size=15)

# Create widgets
# Default color is the color of the points
picker1 = ColorPicker(title="Line Color", color=points.glyph.line_color)
spinner1 = Spinner(title="Circle Size", low=0, high=60, step=5,
                   value=points.glyph.size)  # Default value is the size of the points

# Construct the layout
layout = row(plt, column(picker1, spinner1))

show(layout)

However, these widgets have no effect when used because

it is necessary to define a `CallbackFunction`. We use the simple `jslink` function that acts on the color and size of the points. We add this to our code:


In [8]:

from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.models import ColorPicker, Spinner
from bokeh.layouts import row, column

x = [1, 2, 3, 4, 5, 6]
y = [15, 16, 15, 15, 17, 18]
data = ColumnDataSource(data={'x': x, 'y': y})
plt = figure(title="Application with colors")
points = plt.circle(x='x', y='y', source=data,
                    fill_color=None, line_color='red', size=15)

picker1 = ColorPicker(title="Line Color", color=points.glyph.line_color)
spinner1 = Spinner(title="Circle Size", low=0, high=60,
                   step=5, value=points.glyph.size)

# When picker1 is activated
# Get the selected 'color'
# Use it to update the line_color attribute of points.glyph
picker1.js_link('color', points.glyph, 'line_color')

# When spinner1 is activated
# Get the selected 'value'
# Use it to update the size attribute of points.glyph
spinner1.js_link("value", points.glyph, "size")

layout = row(plt, column(picker1, spinner1))
show(layout)

`js_link` is easy to use but restricted to very specific elements. For more modularity, `Custom_js` can be used.

## 3.2 Custom JavaScript

In the following example, we choose to display a series of different values based on the user's choice: temperatures of Rennes or Paris.

The ColumnDataSource contains all the data, `trennes` and `tparis`, and the Rennes data is initially copied into the `temp` column. We plot a diamond curve of `temp` against `day`.

We create a dropdown menu `menu` that contains two values: Rennes and Paris, associated with 1 and 2, respectively.


In [9]:
from bokeh.models import ColumnDataSource, CustomJS, Dropdown
from bokeh.plotting import figure, show
from bokeh.layouts import row

# Basic data
day = [1, 2, 3, 4, 5, 6]
Trennes = [15, 16, 15, 15, 17, 18]
Tparis = [14, 13, 17, 18, 16, 12]
df = pd.DataFrame({'day': day, 'temp': Trennes,
                  'trennes': Trennes, 'tparis': Tparis})
df

,day,temp,trennes,tparis
0,1,15,15,14
1,2,16,16,13
2,3,15,15,17
3,4,15,15,18
4,5,17,17,16
5,6,18,18,12


In [10]:

source = ColumnDataSource(df)
# Construct the graph
p1 = figure(title="Temperatures")
p1.diamond(x='day', y='temp', source=source, color='blue', size=15)

# Create the widget
menu = Dropdown(label="City Selection", menu=[('Rennes', '1'), ('Paris', '2')])

# Indicate the effect of a click on the menu: it is the code described in CustomJS
# Our goal is to display the data of Rennes or Paris, i.e., replace the values ​​contained in the 'temp' column of the DataFrame.
# If it's Rennes, i.e., the value 1, that is selected, we put the values ​​of 'trennes' in the 'temp' column.
# If it's Paris, i.e., the value 2, that is selected, we put the values ​​of 'tparis' in the 'temp' column.
callback = CustomJS(args=dict(source=source), code="""
    const data = source.data;
    const val = this.item;
    const x = data['day'];
    const y = data['temp'];
    const y1 = data['trennes'];
    const y2 = data['tparis'];
    console.log('Selected value: '+val);  
    if (val == 1) {
        for (let i = 0; i < x.length; i++) {
            y[i] = y1[i];
        }
    } else {
        for (let i = 0; i < x.length; i++) {
            y[i] = y2[i];
        }
    }
    source.change.emit();
""")

# A click on the menu will call the callback code
menu.js_on_event('menu_item_click', callback)

# Construct the layout
layout = row(p1, menu)
show(layout)

## 3.3 Complete Interaction Example

To conclude this course, here is a complete example from the Bokeh documentation. It is a Lasso selector that dynamically traces a line corresponding to the average of the selected points.


In [11]:

from random import random
from bokeh.plotting import figure, show
from bokeh.models import CustomJS, ColumnDataSource

# Random data generation
x = [random() for x in range(500)]
y = [random() for y in range(500)]
d1 = ColumnDataSource(data=dict(x=x, y=y))

# Create a diagram with a selection tool
p = figure(width=500, height=500, tools="lasso_select")

# Plot points
p.circle('x', 'y', color='navy', size=10, alpha=0.4, source=d1,
         selection_color="firebrick", selection_alpha=0.3)

# Plot a horizontal line corresponding to the average, by default set to 0.5
d2 = ColumnDataSource(data=dict(xm=[0, 1], ym=[0.5, 0.5]))
p.line(x='xm', y='ym', color='orange', line_width=5, alpha=0.6, source=d2)

# The code of the callback function indicates:
# when selecting, get the selected points, calculate their average
# Update the d2 plot with this new average
js_code = """
    var inds = d1.selected.indices;
    if (inds.length == 0)
        return;
    var ym=0;
    for (var i = 0; i < inds.length; i++){
        ym += d1.data.y[inds[i]];
    }
    ym /= inds.length;
    d2.data.ym = [ym, ym];
    d2.change.emit();
"""

callback = CustomJS(args=dict(d1=d1, d2=d2), code=js_code)
d1.selected.js_on_change("indices", callback)

show(p)

# 4. Enhancing web pages


Within a Bokeh Layout, two HTML content elements can be added:

- A Paragraph `<p>` tag for inserting text.
- A `<div>` block tag for inserting a block of HTML code.

These elements can be included in Rows or Columns of the Bokeh layout.

Here's an example:


In [12]:

from bokeh.layouts import row, column
from bokeh.models import ColorPicker, Spinner, Div, Paragraph
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
from bokeh.io import output_notebook

# Basic data
x = [1, 2, 3, 4, 5, 6]
y = [15, 16, 15, 15, 17, 18]
data = ColumnDataSource(data={'x': x, 'y': y})

# Construct the graph
plt = figure(title="Application with colors")
points = plt.circle(x='x', y='y', source=data,
                    fill_color=None, line_color='red', size=15)

# Create widgets
picker1 = ColorPicker(title="Line Color", color=points.glyph.line_color)
picker1.js_link('color', points.glyph, 'line_color')
spinner1 = Spinner(title="Circle Size", low=0, high=60,
                   step=5, value=points.glyph.size)
spinner1.js_link("value", points.glyph, "size")

# Add HTML code
div = Div(text="""
<h1> Data Visualization Course 3 </h1>
<p> This course aims to demonstrate how to integrate HTML code</p>
<a href="http://www.univ-rennes2.fr ">A link to the University's website</a>""")

par = Paragraph(text="You can change the rendering of the elements:")

# Construct the layout
layout = column(div, row(plt, column(par, picker1, spinner1)))
show(layout)

In this example, a `div` block was added at the top of the page, containing an `<h1>` title, `<p>` text, and an `<a>` link.

Additionally, a paragraph was added to the right above the widgets.

It's worth noting that it's possible to specify style attributes for the elements. In the following example, a style is applied to the `div` block:


In [13]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show
from bokeh.models import ColorPicker, Spinner, Div, Paragraph
from bokeh.layouts import row, column

# Basic data
x = [1, 2, 3, 4, 5, 6]
y = [15, 16, 15, 15, 17, 18]
data = ColumnDataSource(data={'x': x, 'y': y})

# Construct the graph
plt = figure(title="Application with colors")
points = plt.circle(x='x', y='y', source=data,
                    fill_color=None, line_color='red', size=15)

# Create widgets
picker1 = ColorPicker(title="Line Color", color=points.glyph.line_color)
picker1.js_link('color', points.glyph, 'line_color')
spinner1 = Spinner(title="Circle Size", low=0, high=60,
                   step=5, value=points.glyph.size)
spinner1.js_link("value", points.glyph, "size")

# Add HTML code with styles
div = Div(text="""
<h1> Data Visualization Course 6 </h1>
<p> This course aims to demonstrate how to integrate HTML code</p>
<a href="http://www.univ-rennes2.fr ">A link to the University's website</a>""", styles={'font-style': 'italic', 'color': '#00AAAA'})

par = Paragraph(text="You can change the rendering of the elements:")

# Construct the layout
layout = column(div, row(plt, column(par, picker1, spinner1)))
show(layout)

In [14]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [15]:
import pandas as pd
from bokeh.models import ColumnDataSource
df_waste = pd.read_csv("stats-collecte-dechets.csv", delimiter=";")
df_waste = df_waste.astype({'ANNEE':'int'})
df_waste = df_waste.sort_values('ANNEE')
data_waste = ColumnDataSource(data=df_waste)
df_waste.head(5)

,ANNEE,POPULATION,COLL_DECHETS_ENS,COLL_OM_HAB,COLL_DMREC_HAB,COLL_DECHETERIESENC_HAB,COLL_VEG_HAB
16,2002,385130,181926.000000,301.0,43.0,72.0,56.0
3,2003,392410,180593.640000,266.0,65.0,78.0,52.0
4,2004,396091,187154.065000,236.0,82.0,81.0,73.0
11,2005,399819,186832.463998,231.0,86.0,87.0,62.0
9,2006,403925,190536.803994,224.0,89.0,87.0,72.0


In [16]:
df_waste["ANNEE"] = df_waste["ANNEE"].astype("str")
df_waste_year = df_waste.set_index("ANNEE")
df_waste_year = df_waste_year.loc[:, ~df_waste_year.columns.isin(["POPULATION", "COLL_DECHETS_ENS"])].T

color = ["grey", "yellow", "orange", "green"]
label = ["Household Waste", "Recyclable Waste", "Recycling Center", "Green Waste"]
value = df_waste_year.loc[:, "2021"]

df_waste_year['Color'] = color
df_waste_year['Label'] = label
df_waste_year['Value'] = value


data_waste_year = ColumnDataSource(data=df_waste_year)

df_waste_year

ANNEE,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,...,2015,2016,2017,2018,2019,2020,2021,Color,Label,Value
COLL_OM_HAB,301.0,266.0,236.0,231.0,224.0,216.0,228.0,224.0,216.0,215.0,...,202.0,199.0,192.0,188.0,183.0,180.0,181.0,grey,Household Waste,181.0
COLL_DMREC_HAB,43.0,65.0,82.0,86.0,89.0,89.0,96.0,96.0,94.0,96.0,...,89.0,90.0,93.0,97.0,99.0,98.0,101.0,yellow,Recyclable Waste,101.0
COLL_DECHETERIESENC_HAB,72.0,78.0,81.0,87.0,87.0,93.0,99.0,101.0,107.0,109.0,...,116.0,119.0,116.0,119.0,121.0,117.0,137.0,orange,Recycling Center,137.0
COLL_VEG_HAB,56.0,52.0,73.0,62.0,72.0,84.0,83.0,86.0,68.0,73.0,...,71.0,69.0,65.0,64.0,66.0,56.0,67.0,green,Green Waste,67.0


In [17]:
from bokeh.models import Tabs, TabPanel, ColumnDataSource, CustomJS, Select
from bokeh.palettes import Set2

from bokeh.layouts import row, column
from bokeh.models import ColorPicker, Spinner, Div, Paragraph, NumeralTickFormatter, TableColumn, DataTable, HoverTool
from bokeh.plotting import figure, show

p1 = figure(title="Collected Waste in Rennes Métropole")
data_column_names_p1 = ["COLL_OM_HAB", "COLL_DMREC_HAB", "COLL_VEG_HAB", "COLL_DECHETERIESENC_HAB"]
data_legend_label_p1 = ["Household Waste", "Recyclable Waste", "Recycling Center", "Green Waste"]

for column_name, legend_label, color in zip(data_column_names_p1, data_legend_label_p1, Set2[4]):
    p1.line("ANNEE", column_name, source=data_waste, line_width=5, color=color, alpha=0.8, legend_label=legend_label,
            muted_color=color, muted_alpha=0.2)

p1.legend.click_policy = "hide"

p2 = figure(title="Population evolution of Rennes Métropole")
lines_p2 = p2.line('ANNEE', 'POPULATION', color='red', source=data_waste)
p2.yaxis.formatter = NumeralTickFormatter(format="0")
picker_p2 = ColorPicker(title="Line Color", color=lines_p2.glyph.line_color)
picker_p2.js_link('color', lines_p2.glyph, 'line_color')
spinner_p2 = Spinner(title="Line Thickness", low=0, high=20, value=lines_p2.glyph.line_width)
spinner_p2.js_link("value", lines_p2.glyph, "line_width")
div_p2 = Div(text="""
<p>Customize the appearence of the population evolution plot</p>
""")

layout_p2 = column(p2, div_p2, row(picker_p2, spinner_p2))

layout_tab1 = row([p1, layout_p2])



tab1 = TabPanel(child=layout_tab1, title="Total Waste Evolution")


p3 = figure(x_range=data_waste_year.data['Label'], toolbar_location=None, tools="", tooltips="@Label: @Value kg/inhabitant")
p3.vbar(x='Label', top='Value', width=0.5, color='Color', source=data_waste_year)

year_tuples = [(str(year), str(year)) for year in df_waste["ANNEE"].unique()]

menu_p3 = Select(title="Choose the Year", value=year_tuples[-1][0], options=year_tuples)

# Update the CustomJS callback for the Select widget
callback_p3 = CustomJS(args=dict(source=data_waste_year), code="""
    const data = source.data;
    const selected_year = cb_obj.value;  // Use cb_obj.value for Select widget
    const labels = data['Label'];
    
    for (let i = 0; i < labels.length; i++) {
        data['Value'][i] = data[selected_year][i];
    }

    source.change.emit();
""")

# Attach the callback to the Select widget
menu_p3.js_on_change('value', callback_p3)

# # Create a HoverTool instance
# hover_tool = HoverTool(tooltips=[
#     ('Category', '@Label'),  # Display the label/category
#     ('Value', '@Value')      # Display the corresponding value
# ])
# p3.add_tools(hover_tool)

columns = [
        TableColumn(field="Label", title="Category"),
        TableColumn(field="Value", title="Weight per inhabitant (kg)"),
    ]
data_table = DataTable(source=data_waste_year, columns=columns)


layout_tab2 = column(menu_p3, row(p3, data_table))
tab2 = TabPanel(child=layout_tab2, title="Waste per inhabitant")


tabs = Tabs(tabs=[tab1, tab2])

div_main = Div(text="""
<h1> Waste collection in Rennes Metropole from 2002 to 2022 </h1>
<p> This data is provided by 
<a href="https://data.rennesmetropole.fr/explore/dataset/stats-collecte-dechets/table/"> Rennes Metropole open data</a> </p>""")


layout_main = column(div_main, tabs)

show(layout_main)